In [1]:
# spliting .py into cells and locally test by Jupyter Notebook 
!pip install pandas dash

In [2]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

C:\Users\mapleseason\AppData\Local\Temp\ipykernel_12788\1399112611.py:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
C:\Users\mapleseason\AppData\Local\Temp\ipykernel_12788\1399112611.py:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


In [3]:
# Read the airline data into pandas dataframe
spacex_csv_file = "D://PERSONAL//DataEngine//IBM_DS//10_Capstone//spacex_launch_dash.csv"
spacex_df = pd.read_csv(spacex_csv_file)
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [4]:
# Create a dash application
app = dash.Dash(__name__)
app.config.suppress_callback_exceptions = True
# Clear the layout and do not display exception till callback gets executed 

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign':'center','color':'#503D36','font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(id='site-dropdown',
                                             options=[
                                                     {'label': 'All Sites','value':'ALL'},
                                                     {'label': 'CCAFS LC-40','value':'CCAFS LC-40'},
                                                     {'label': 'CCAFS SLC-40','value':'CCAFS SLC-40'},
                                                     {'label': 'KSC LC-39A','value':'KSC LC-39A'},
                                                     {'label': 'VAFB SLC-4E','value':'VAFB SLC-4E'}
                                                     ],
                                             value='ALL',
                                             placeholder='Select a launch site here:',
                                             searchable=True,
#                                              style={'width':'80%','padding':'3px','font-size':'20px','text-align-last':'center'}
                                             ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                # Example cited from https://dash.plotly.com/dash-core-components/rangeslider
                                # app.layout = html.Div([
                                #     dcc.RangeSlider(0, 20, 1, value=[5, 15], id='my-range-slider'),
                                #     html.Div(id='output-container-range-slider')
                                # ])
                                dcc.RangeSlider(id='payload-slider',
                                                min=0,max=10000,step=1000,value=[min_payload, max_payload]
                                               ),
                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])


In [5]:
# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
                Input(component_id='site-dropdown', component_property='value'))

def get_graph(in_site):
    if in_site == 'ALL':
        pie_fig = px.pie(spacex_df, values='class', names='Launch Site', title='Total Success Launches by Site')
        return pie_fig
    else:
        pie_df = spacex_df[spacex_df['Launch Site']==in_site].groupby(['Launch Site','class']).size().reset_index(name='Counts')
        pie_fig = px.pie(pie_df, values='Counts', names='class', title=f"Total Success Launches for Site {in_site}")
        return pie_fig

In [6]:
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart',component_property='figure'),
                Input(component_id='site-dropdown', component_property='value'),
                Input(component_id='payload-slider', component_property='value'))

def get_scatter(in_site, in_payload):
    
    if in_site == 'ALL':
        sca_fig = px.scatter(spacex_df[spacex_df['Payload Mass (kg)'].between(in_payload[0],in_payload[1])],
                             x='Payload Mass (kg)',y='class', color='Booster Version Category', title='Correlation between Payload and Success for all Sites')
        return sca_fig
    else:
        sca_fig = px.scatter(spacex_df[spacex_df['Launch Site']==in_site],
                             x='Payload Mass (kg)',y='class', color='Booster Version Category', title=f"Correlation between Payload and Success for Sites of {in_site}")
        return sca_fig

In [ ]:
# Run the app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [28/Aug/2022 11:49:55] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Aug/2022 11:49:55] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [28/Aug/2022 11:49:55] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [28/Aug/2022 11:49:55] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [28/Aug/2022 11:49:55] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [28/Aug/2022 11:49:55] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 200 -
127.0.0.1 - - [28/Aug/2022 11:49:55] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [28/Aug/2022 11:49:56] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/Aug/2022 11:49:56] "POST /_dash-update-component HTTP/1.1" 200 -
